In [27]:
import pandas as pd

from pipeline.service import FileService

# the first file in the raw-data folder
file_path = '../data/2024-09-13 15:29 Test-Load-inspect dataset/raw-data/labse-three-year-optimized.tsv'

dataset = FileService.read_tsv_to_df(file_path=file_path)

dataset.info()
dataset.describe(include='all')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [2]:
dataset

,id,pubtime,medium_code,medium_name,rubric,regional,doctype,doctype_description,language,char_count,dateline,head,subhead,article_link,content_id,content
0,38899273,2020-02-10 08:28:00+01,ZWAO,20 minuten online,NaN,NaN,WWE,Online medium,de,562,Singapore Airlines,A380 muss wegen Orkantief durchstarten,NaN,https://www.20min.ch/schweiz/zuerich/story/242...,00028ac0-46af-cb40-908b-dedebaf3a9ea,<tx><ld><p>Sturmtief Sabine wirbelt den Plan d...
1,39289731,2020-04-01 16:55:00+02,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,697,Genève,La police interrompt la chorale au balcon,NaN,http://www.20min.ch/ro/news/geneve/story/31051453,00041b4b-dee5-c672-7218-3b6a9690cf4c,<tx><ld><p>Les voisins chanteurs de la rotonde...
2,39297090,2020-04-02 12:54:31+02,ZWAO,20 minuten online,NaN,NaN,WWE,Online medium,de,2334,Durchsetzung der Verbote,Aargauer Polizei setzt auf Live-Überwachung,NaN,https://www.20min.ch/schweiz/news/story/20300676,00052c9a-566b-b931-d1e2-30c94d187cc3,<tx><ld><p>Der Kanton Aargau führt im Kampf ge...
3,39212766,2020-03-20 21:40:00+01,ZWAO,20 minuten online,NaN,NaN,WWE,Online medium,de,3773,Intermeccanica Murena 429 GT,Ein exzentrischer Express-Kombi aus Italien,NaN,http://www.20min.ch/motor/youngtimer/story/258...,0005c8c7-18c1-5868-6bb6-e392cc4bc82b,<tx><ld><p>Inmitten der anderen Automobile der...
4,39266324,2020-03-28 20:38:00+01,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,2846,Coronavirus,La France commande un milliard de masques,NaN,http://www.20min.ch/ro/news/monde/story/31929765,000668c6-3800-6d53-923a-def1b319d481,<tx><ld><p>Le ministre de la santé Olivier Vér...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23550,38678500,2020-01-13 13:35:00+01,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,1660,Fribourg,La collecte de sapins de Noël à chevaux a séduit,NaN,https://www.20min.ch/ro/news/romandie/story/15...,ffe8cc26-18cd-8c02-e699-86492530552d,<tx><ld><p>La collecte de sapins de Noël a été...
23551,38998370,2020-02-22 06:06:00+01,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,2044,Suisse,Affaire Crypto: la Suisse interpellée par des ...,NaN,http://www.20min.ch/ro/news/suisse/story/27869743,ffe95c5d-0fda-3f4b-b496-780410fb1875,<tx><ld><p>Un porte-parole du Département fédé...
23552,39357903,2020-04-12 11:03:00+02,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,1429,Valais,"Accidentée, une voiture met le feu à la forêt",NaN,http://www.20min.ch/ro/news/romandie/story/306...,ffee4610-9e0a-f485-7fe6-614d4bb5b9bc,<tx><ld><p>L'incendie d'un véhicule s'est prop...
23553,39194583,2020-03-18 15:02:35+01,ZWSO,20 minutes online,NaN,NaN,WWE,Online medium,fr,1124,Coronavirus,Le concours Eurovision de la chanson est annulé,NaN,https://www.20min.ch/ro/news/monde/story/29637867,ffef5d86-5088-8009-8d1f-999d93e38e56,<tx><ld><p>Comme l'épidémie de Covid-19 se pou...


In [3]:
########## Date range #########
dataset['pubtime'] = pd.to_datetime(dataset['pubtime'], utc=True)
date_range = (dataset['pubtime'].min(), dataset['pubtime'].max())
date_range_formatted = f"{dataset['pubtime'].min().strftime('%Y-%m-%d')} - {dataset['pubtime'].max().strftime('%Y-%m-%d')}"

date_range_formatted

'2020-01-01 - 2020-06-30'

In [4]:
##### Articles per month ######
articles_per_month = dataset['pubtime'].dt.to_period('M').value_counts().sort_index()

articles_per_month

/var/folders/h7/fxdx1q0j1mv0gdt_m2xm54jc0000gn/T/ipykernel_55975/1427310849.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  articles_per_month = dataset['pubtime'].dt.to_period('M').value_counts().sort_index()


pubtime
2020-01    5740
2020-02    5320
2020-03    4894
2020-04    4216
2020-05     559
2020-06    2826
Freq: M, Name: count, dtype: int64

In [5]:
int(articles_per_month.mean())

3925

In [6]:
total_articles = len(dataset)

language_counts = dataset["language"].value_counts()

language_ratio = language_counts / total_articles

language_counts.to_dict()
language_ratio.to_dict()

{'fr': 0.6585013797495224, 'de': 0.3414986202504776}

In [7]:
import numpy as np
# Calculate word count statistics
word_count_stats = {
 'mean': dataset['char_count'].mean(),
 'median': dataset['char_count'].median(),
 'range': (dataset['char_count'].min(), dataset['char_count'].max())
}
word_count_stats


{'mean': 2428.6164296327743, 'median': 2129.0, 'range': (164, 42844)}

In [8]:
# Distribution of articles by word count range
bins = [0, 500, 1000, 1500, np.inf]
labels = ['<500', '500-1000', '1000-1500', '>1500']
dataset['word_count_category'] = pd.cut(dataset['char_count'], bins=bins, labels=labels)
word_count_distribution = dataset['word_count_category'].value_counts()
word_count_distribution

word_count_category
>1500        17137
1000-1500     3796
500-1000      2452
<500           170
Name: count, dtype: int64

In [9]:
table_one = {
    "Publication date range": date_range_formatted,
    "Number of articles": total_articles,
    "German": str(language_counts['de']) + ' - ' + f"{language_ratio['de'] * 100:.2f}%",
    "French": str(language_counts['fr']) + ' - ' + f"{language_ratio['fr'] * 100:.2f}%",
    "Content statistics" : {
        "Mean # of words": int(word_count_stats['mean']),
        "Median # of words" : int(word_count_stats['median']),
        "Min # of words": word_count_stats['range'][0],
        "Max # of words": word_count_stats['range'][1]
    },
    "Word count distribution": word_count_distribution.to_dict()
}

table_one

{'Publication date range': '2020-01-01 - 2020-06-30',
 'Number of articles': 23555,
 'German': '8044 - 34.15%',
 'French': '15511 - 65.85%',
 'Content statistics': {'Mean # of words': 2428,
  'Median # of words': 2129,
  'Min # of words': 164,
  'Max # of words': 42844},
 'Word count distribution': {'>1500': 17137,
  '1000-1500': 3796,
  '500-1000': 2452,
  '<500': 170}}

In [11]:
######## formatting ############
content_stats_df = pd.DataFrame.from_dict(table_one["Content statistics"], orient='index', columns=['Value'])
word_count_dist_df = pd.DataFrame.from_dict(table_one["Word count distribution"], orient='index', columns=['Count'])

In [10]:
########## CONVERT TO HTML ##################
content_stats_html = content_stats_df.to_html(header=False).replace('\n', '').replace('  ', '')
word_count_dist_html = word_count_dist_df.to_html(header=False).replace('\n', '').replace('  ', '')

main_data = {
    "Publication date range": [table_one["Publication date range"]],
    "Number of articles": [table_one["Number of articles"]],
    "German": [table_one["German"]],
    "French": [table_one["French"]],
    "Content statistics": [content_stats_html],
    "Word count distribution": [word_count_dist_html]
}

main_df = pd.DataFrame(main_data)
transposed_df = main_df.transpose()

html_table = transposed_df.to_html(escape=False, header=False)
html_table = html_table.replace("\n", '')

with open('table_one.html', 'w') as file:
    file.write(html_table)

In [26]:
################# CONVERT TO LATEX #############
main_data = {
    "Publication date range": [table_one["Publication date range"]],
    "Number of articles": [table_one["Number of articles"]],
    "German": [table_one["German"].replace("%", "\\%")],
    "French": [table_one["French"].replace("%", "\\%")],
    "Content statistics": [content_stats_df.to_latex(header=False, index=True)],
    "Word count distribution": [word_count_dist_df.to_latex(header=False, index=True)]
}

main_df = pd.DataFrame(main_data)
transposed_df = main_df.transpose()

latex_table = transposed_df.to_latex(escape=False, header=False, column_format='|l|r|', bold_rows=True, label='Table 1: dataset statistics')
latex_table = latex_table.replace('#', '\\#')

# Save the LaTeX table to a file
with open('table_one.tex', 'w') as file:
    file.write(latex_table)